In [0]:
import os, sys, json
sys.path.append(os.path.abspath("../modules/"))

from petl.ingestion import read_volume_csv

target_schema = dbutils.widgets.get("target_schema")
entity = json.loads(dbutils.widgets.get("entity"))

if entity['source_type'] == 'volume_csv':
    max_watermark = None
    if entity['watermark'] != "null":
        try:
            max_watermark = spark.sql(f"""
                SELECT 
                    MAX({entity['watermark']})
                FROM bronze.{target_schema}.{entity['entity']}
            """).collect()[0][0]
        except:
            pass

    df = read_volume_csv(spark, entity['connection_details']['volume_path'], entity['watermark'], max_watermark)

df.write.mode("append").option("mergeSchema", True).saveAsTable(f"bronze.{target_schema}.{entity['entity']}")
